In [1]:
import os
import json
import requests
import pandas as pd

from sqlalchemy import create_engine
from time import perf_counter, sleep
from datetime import datetime, timedelta

import config_bestbuy

In [10]:
def initialize(folder_index=0):

    folders = ['products', 'categories', 'stores', 'products_update']
    datename = datetime.utcnow().strftime('%Y%m%d')
    path = config_bestbuy.path
    foldername = f'best_buy_{datename}\\{folders[folder_index]}'
    folderpath = os.path.join(path, foldername)

    if not os.path.exists(folderpath):
        os.makedirs(folderpath)

    db = os.path.join(config_bestbuy.path, 'bestbuy.db')
    conn_string = f'sqlite:///{db}'
    engine = create_engine(conn_string)

    with engine.connect() as cnx:
        try:
            sel_stmt = "SELECT * FROM products LIMIT 0"
            df_db = pd.read_sql(sql=sel_stmt, con=cnx)
            db_cols = df_db.columns.tolist()

            last_update_stmt = 'SELECT MAX(itemUpdateDate) FROM products'
            df_itemUpdateDate = pd.read_sql(sql=last_update_stmt, con=cnx)
            last_update_date = df_itemUpdateDate.iloc[0, 0]
        except Exception as e:
            db_cols = []
            
    return folderpath, datename, engine, db_cols, last_update_date


def api_bestbuy(cursorMark="*", api_index=0, page_size=100, last_update_date=None):
    key = config_bestbuy.key_bestbuy
    apis = ['products', 'categories', 'stores', f'products(itemUpdateDate>{last_update_date}&active=*)']
    url = f"https://api.bestbuy.com/v1/{apis[api_index]}"
    payload = {
        'apiKey': key, 
        'pageSize': page_size, 
        'format': 'json', 
        'show': 'all',
        'cursorMark': cursorMark
        }
    r = requests.get(f'{url}', params=payload)

    return r


def insert_db(r, engine, db_cols):

    with engine.connect() as cnx:
        cols = ['nextCursorMark', 'total', 'totalPages', 'queryTime', 'totalTime', 'canonicalUrl', 'sku', 'name', 'type', 'startDate', 'new', 'activeUpdateDate', 'active', 'regularPrice', 'salePrice', 'clearance', 'onSale', 'categoryPath', 'customerReviewCount', 'customerReviewAverage', 'priceUpdateDate', 'itemUpdateDate', 'class', 'classId', 'subclass', 'subclassId', 'department', 'departmentId', 'images', 'image', 'color']
        io = r.json()
        df_meta = pd.DataFrame(io)
        df_meta = df_meta.iloc[:, :-1]
        df_products = pd.DataFrame(io['products'])
        df = df_meta.merge(df_products, how='inner', left_index=True, right_index=True)
        df = df.loc[:, cols]
        df.insert(0, 'request_timestamp', datetime.utcnow())

        for col in df.columns.tolist():
            if df[col].dtype == 'object' or df[col].dtype == 'int64':
                df[col] = df[col].astype('str')

            datatype = df[col].dtype
            if datatype == 'bool':
                datatype = 'INTEGER'
            if datatype == 'float64':
                datatype = 'REAL'
            elif datatype == 'datetime64[ns]':
                datatype = 'NUMERIC'
            else: # int64, object, TEXT
                datatype = 'TEXT'
            
            if col not in db_cols:
                alter_stmt = f"""
                ALTER TABLE products
                ADD {col} {datatype};
                """
                try:
                    cnx.execute(alter_stmt)

                except Exception as e:
                    print(f'error: {col} already exists')
                
                finally:
                    db_cols.append(col)
                
        df.to_sql(name='products', con=cnx, if_exists='append', index=False)


def main(api_index=0, page_size=100, db=False):

    t0 = perf_counter()
    folderpath, datename, engine, db_cols, last_update_date = initialize(folder_index=api_index)
    nextcursorMark = "*"
    pg = 0
    pages = api_bestbuy(cursorMark=nextcursorMark, api_index=api_index, page_size=page_size, last_update_date=last_update_date).json()['totalPages']
    print(f'{pages=}')
    sleep(1)
    t1 = perf_counter()

    while True:
        print(f'{pg=}', f'{nextcursorMark=}', f'{t1=}', f'{perf_counter()-t1=}', sep=' | ')
        r = api_bestbuy(cursorMark=nextcursorMark, api_index=api_index, page_size=page_size, last_update_date=last_update_date)

        if r.status_code == 200:
            try:
                if len(r.json()['products']) == 0:
                    print(f"fin: {(perf_counter()-t0)=}")
                    break      

                nextcursorMark = r.json()['nextCursorMark']
                pg += 1
            
            except (NameError, ValueError, KeyError) as e:
                print(f'{e=}')
                t1 = perf_counter()

            else:
                pass

            finally:
                pass
            
            filename = f'best_buy_{datename}_{pg:05}.json'
            filepath = os.path.join(folderpath, filename)

            with open(filepath, 'w') as f:
                json.dump(r.json(), f, indent=4)

            if db:
                insert_db(r=r, engine=engine, db_cols=db_cols)

        timer = perf_counter()-t1
        if (pg)%5==0:
            if timer<1:
                sleep(1-timer)
            t1 = perf_counter()
    


In [11]:
if __name__ == '__main__':

    # api_index = {0: 'products', 1: 'categories', 2: 'stores', 3: 'products(itemUpdateDate>today&active=*)'}
    main(api_index=3, page_size=100, db=True)


pages=43
pg=0 | nextcursorMark='*' | t1=351.887903 | perf_counter()-t1=2.340000003187015e-05
pg=1 | nextcursorMark='AoIIP4AAADNwcm9kdWN0XzM1MDMwMjI1X3Vz' | t1=351.887903 | perf_counter()-t1=1.980381599999987
pg=2 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzQ4ODE5MDFfdXM=' | t1=351.887903 | perf_counter()-t1=4.56571409999998
pg=3 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzU3MTIxMTBfdXM=' | t1=351.887903 | perf_counter()-t1=6.974869300000023
pg=4 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzU4MDI5MDNfdXM=' | t1=351.887903 | perf_counter()-t1=9.047557100000006
pg=5 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzYwMjE0MjdfdXM=' | t1=363.5691262 | perf_counter()-t1=1.2199999957829277e-05
pg=6 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzYyMzU0MDBfdXM=' | t1=363.5691262 | perf_counter()-t1=2.584062999999958
pg=7 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzYyODc3MjlfdXM=' | t1=363.5691262 | perf_counter()-t1=5.041632399999969
pg=8 | nextcursorMark='AoIIP4AAADJwcm9kdWN0XzYzMTc0NTNfdXM=' | t1=363.5691262 | perf_counter()-

Tip: To query for updates or deltas since you last walked through the result set you can use the itemUpdateDate attribute. To ensure that your query results include changes to a product’s active/inactive status, add active=* to your query parameters. 
For example: .../v1/products(itemUpdateDate>2017-02-06T16:00:00&active=*)?format=json&pageSize=100&cursorMark=*&apiKey=YOUR_API_KEY
For example: .../v1/products(itemUpdateDate>today&active=*)?format=json&pageSize=100&cursorMark=*&apiKey=YOUR_API_KEY
"https://api.bestbuy.com/v1/products(releaseDate>today)?format=json&show=sku,name,salePrice&apiKey=YourAPIKey"

In [79]:
bb_root = config_bestbuy.path
bb_dir = 'best_buy_20211123'
bb_path = os.path.join(bb_root, bb_dir)

bb_filename = 'best_buy_20211123_00001 - Copy.json'
bb_file_path = os.path.join(f'{bb_path}\\products', bb_filename)

with open(bb_file_path, 'r') as f:
    io = json.load(f)


C:\Users\Admin\Downloads\best_buy\best_buy_20211123


In [7]:
nextcursorMark = "*"
api_index = 3
page_size = 100
days = 2

r = api_bestbuy(cursorMark=nextcursorMark, api_index=api_index, page_size=page_size, days=days)
# io = r.json()


In [9]:
cols = ['nextCursorMark', 'total', 'totalPages', 'queryTime', 'totalTime', 'canonicalUrl', 'sku', 'name', 'type', 'startDate', 'new', 'activeUpdateDate', 'active', 'regularPrice', 'salePrice', 'clearance', 'onSale', 'categoryPath', 'customerReviewCount', 'customerReviewAverage', 'priceUpdateDate', 'itemUpdateDate', 'class', 'classId', 'subclass', 'subclassId', 'department', 'departmentId', 'images', 'image', 'color']
io = r.json()
df_meta = pd.DataFrame(io)
df_meta = df_meta.iloc[:, :-1]
df_products = pd.DataFrame(io['products'])
df = df_meta.merge(df_products, how='inner', left_index=True, right_index=True)
df = df.loc[:, cols]
df.insert(0, 'request_timestamp', datetime.utcnow())

In [12]:
api_index=3
folderpath, datename, engine, db_cols, last_update_date = initialize(folder_index=api_index)
last_update_date

'2021-11-26T23:48:12'

In [19]:

db = os.path.join(config_bestbuy.path, 'bestbuy.db')
conn_string = f'sqlite:///{db}'
engine = create_engine(conn_string)

with engine.connect() as cnx:
    # df.to_sql(name='products', con=cnx, if_exists='replace', index=False)
    df_test = pd.read_sql(sql='select max(itemUpdateDate) from products', con=cnx)
    rs = cnx.execute("select max(itemUpdateDate) from products")
    for row in rs:
        print(row)


('2021-11-26T23:00:31',)


In [20]:
df_test.iloc[0, 0]

'2021-11-26T23:00:31'

In [231]:
today = datetime.utcnow().date()
n_days = datetime.utcnow().date() - timedelta(days=2)
n_hours = (today-n_days)/timedelta(seconds=1)
n_hours

172800.0

In [ ]:
['nextCursorMark', 'total', 'totalPages', 'queryTime', 'totalTime', 'canonicalUrl', 'sku', 'name', 'type', 'startDate', 'new', 'activeUpdateDate', 'active', 'regularPrice', 'salePrice', 'clearance', 'onSale', 'categoryPath', 'customerReviewCount', 'customerReviewAverage', 'priceUpdateDate', 'itemUpdateDate', 'class', 'classId', 'subclass', 'subclassId', 'department', 'departmentId', 'images', 'image', 'color'] 
